In [1]:
'''
Group members:
Bohao XU        ETU20211498
Jijun TAN       ETU20211472
Yilin Zhang     ETU20211520
Junxin Huang    ETU20211420
'''

'\nGroup members:\nBohao XU        ETU20211498\nJijun TAN       ETU20211472\nYilin Zhang     ETU20211520\nJunxin Huang    ETU20211420\n'

In [2]:
'''
README!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

1. please download the dataset from:
https://www.kaggle.com/competitions/g-research-crypto-forecasting/data

2. please make sure all the files downloaded are placed in the same folder as all the files I submitted to you.

3. then you can run the Model_Training.ipynb and Model_Inference.ipynb
'''

'\nREADME!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n\n1. please download the dataset from:\nhttps://www.kaggle.com/competitions/g-research-crypto-forecasting/data\n\n2. please make sure all the files downloaded are placed in the same folder as all the files I submitted to you.\n\n3. then you can run the Model_Training.ipynb and Model_Inference.ipynb\n'

In [3]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import time
import datetime
from datetime import datetime
import pickle
import gc
import csv
from tqdm import tqdm
import optuna
import warnings

seed0 = 8586

creator='group4'
TRAIN_CSV = 'train.csv'
SUPPLE_TRAIN_CSV = 'supplemental_train.csv'
ASSET_DETAILS_CSV = 'asset_details.csv' 
ASSET_ID_SELECTED_FOR_INFERENCE = [i for i in range(14)]
FEATURES_EXCLUDE = []
lags = [60,300,900]

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', 350)
pd.set_option('mode.chained_assignment', None)

# Data_Cleaning

In [4]:
df_main = pd.read_csv('train.csv', usecols=['timestamp','Asset_ID', 'Close', 'Target','Volume'])
df_supple = pd.read_csv('supplemental_train.csv', usecols=['timestamp','Asset_ID', 'Close', 'Target','Volume'])
# Merge training data from 2 tables
df_train = pd.concat([df_main, df_supple])

In [5]:
# Reconstruct all training data with timestamps as unique columns, ensuring that each row is only 1 timestamp and contains 14 ids of close,target,volume data
train_merged = pd.DataFrame()
train_merged[df_train.columns] = 0
for id in tqdm( range(14) ):
    train_merged = train_merged.merge(df_train.loc[df_train["Asset_ID"] == id, ['timestamp', 'Close','Target','Volume']].copy(), on="timestamp", how='outer',suffixes=['', "_"+str(id)])

100%|██████████| 14/14 [00:33<00:00,  2.37s/it]


In [6]:
# Delete columns from the original df before refactoring
train_merged = train_merged.drop(df_train.columns.drop("timestamp"), axis=1)
train_merged = train_merged.sort_values('timestamp', ascending=True)
train_merged

,timestamp,Close_0,Target_0,Volume_0,Close_1,Target_1,Volume_1,Close_2,Target_2,Volume_2,Close_3,Target_3,Volume_3,Close_4,Target_4,Volume_4,Close_5,Target_5,Volume_5,Close_6,Target_6,Volume_6,Close_7,Target_7,Volume_7,Close_8,Target_8,Volume_8,Close_9,Target_9,Volume_9,Close_10,Target_10,Volume_10,Close_11,Target_11,Volume_11,Close_12,Target_12,Volume_12,Close_13,Target_13,Volume_13
0,1514764860,8.530000,-0.014399,78.380000,13850.176000,-0.014643,31.550062,2374.590000,-0.004218,19.233005,NaN,NaN,NaN,NaN,NaN,NaN,7.657600,-0.013922,6626.713370,738.507500,-0.004809,335.987856,25.877000,-0.008264,121.087310,NaN,NaN,NaN,225.206667,-0.009791,411.896642,NaN,NaN,NaN,329.4600,NaN,6.635710,NaN,NaN,NaN,NaN,NaN,NaN
1,1514764920,8.514500,-0.015875,71.390000,13828.102000,-0.015037,31.046432,2372.286667,-0.004079,24.050259,NaN,NaN,NaN,NaN,NaN,NaN,7.656700,-0.014534,3277.475494,738.260000,-0.004441,232.793141,25.897000,-0.029902,1.468019,NaN,NaN,NaN,224.856667,-0.012991,3640.502706,NaN,NaN,NaN,329.4500,-0.009690,0.349420,NaN,NaN,NaN,NaN,NaN,NaN
2,1514764980,8.484800,-0.015410,1546.820000,13801.314000,-0.010309,55.061820,2372.063333,-0.002892,42.676438,NaN,NaN,NaN,NaN,NaN,NaN,7.651200,-0.012546,5623.557585,737.502500,-0.004206,174.138031,26.469500,-0.030832,76.163922,NaN,NaN,NaN,226.000000,-0.003572,328.350286,NaN,NaN,NaN,329.1900,0.006567,1.189553,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122476,1642982280,384.201667,NaN,496.467537,36276.223333,NaN,49.867700,302.430000,NaN,198.196485,1.123465,NaN,430719.654610,0.141800,NaN,7.433686e+05,2.272983,NaN,14907.125055,2541.388571,NaN,821.614133,25.152780,NaN,1334.601131,0.814233,NaN,20511.345658,112.258000,NaN,790.632440,1831.573333,NaN,2.142748,156.5125,NaN,21.597886,0.200803,NaN,7.076212e+05,0.057509,NaN,1.301079e+06
2122477,1642982340,384.002000,NaN,368.684921,36276.623333,NaN,43.030556,302.562500,NaN,76.954035,1.124831,NaN,352929.794282,0.141800,NaN,7.287195e+05,2.275100,NaN,14559.759824,2541.040000,NaN,749.548301,25.135800,NaN,2410.205214,0.815833,NaN,9064.791667,112.342400,NaN,331.208442,1833.186000,NaN,3.985169,156.8940,NaN,51.115980,0.200968,NaN,1.988516e+05,0.057521,NaN,9.245588e+05
2122478,1642982400,382.930000,NaN,1490.571077,36221.987143,NaN,110.053151,302.112000,NaN,209.645675,1.121151,NaN,249923.229256,0.141391,NaN,1.009688e+06,2.264800,NaN,202194.015531,2536.151429,NaN,1057.765667,25.028067,NaN,6709.816310,0.812400,NaN,6019.269834,111.983333,NaN,1243.860266,1833.018333,NaN,4.190290,156.5500,NaN,74.397471,0.200093,NaN,2.983796e+06,0.057478,NaN,8.988253e+05


In [7]:
train_merged = train_merged.set_index("timestamp")
train_merged

,Close_0,Target_0,Volume_0,Close_1,Target_1,Volume_1,Close_2,Target_2,Volume_2,Close_3,Target_3,Volume_3,Close_4,Target_4,Volume_4,Close_5,Target_5,Volume_5,Close_6,Target_6,Volume_6,Close_7,Target_7,Volume_7,Close_8,Target_8,Volume_8,Close_9,Target_9,Volume_9,Close_10,Target_10,Volume_10,Close_11,Target_11,Volume_11,Close_12,Target_12,Volume_12,Close_13,Target_13,Volume_13
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1514764860,8.530000,-0.014399,78.380000,13850.176000,-0.014643,31.550062,2374.590000,-0.004218,19.233005,NaN,NaN,NaN,NaN,NaN,NaN,7.657600,-0.013922,6626.713370,738.507500,-0.004809,335.987856,25.877000,-0.008264,121.087310,NaN,NaN,NaN,225.206667,-0.009791,411.896642,NaN,NaN,NaN,329.4600,NaN,6.635710,NaN,NaN,NaN,NaN,NaN,NaN
1514764920,8.514500,-0.015875,71.390000,13828.102000,-0.015037,31.046432,2372.286667,-0.004079,24.050259,NaN,NaN,NaN,NaN,NaN,NaN,7.656700,-0.014534,3277.475494,738.260000,-0.004441,232.793141,25.897000,-0.029902,1.468019,NaN,NaN,NaN,224.856667,-0.012991,3640.502706,NaN,NaN,NaN,329.4500,-0.009690,0.349420,NaN,NaN,NaN,NaN,NaN,NaN
1514764980,8.484800,-0.015410,1546.820000,13801.314000,-0.010309,55.061820,2372.063333,-0.002892,42.676438,NaN,NaN,NaN,NaN,NaN,NaN,7.651200,-0.012546,5623.557585,737.502500,-0.004206,174.138031,26.469500,-0.030832,76.163922,NaN,NaN,NaN,226.000000,-0.003572,328.350286,NaN,NaN,NaN,329.1900,0.006567,1.189553,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642982280,384.201667,NaN,496.467537,36276.223333,NaN,49.867700,302.430000,NaN,198.196485,1.123465,NaN,430719.654610,0.141800,NaN,7.433686e+05,2.272983,NaN,14907.125055,2541.388571,NaN,821.614133,25.152780,NaN,1334.601131,0.814233,NaN,20511.345658,112.258000,NaN,790.632440,1831.573333,NaN,2.142748,156.5125,NaN,21.597886,0.200803,NaN,7.076212e+05,0.057509,NaN,1.301079e+06
1642982340,384.002000,NaN,368.684921,36276.623333,NaN,43.030556,302.562500,NaN,76.954035,1.124831,NaN,352929.794282,0.141800,NaN,7.287195e+05,2.275100,NaN,14559.759824,2541.040000,NaN,749.548301,25.135800,NaN,2410.205214,0.815833,NaN,9064.791667,112.342400,NaN,331.208442,1833.186000,NaN,3.985169,156.8940,NaN,51.115980,0.200968,NaN,1.988516e+05,0.057521,NaN,9.245588e+05
1642982400,382.930000,NaN,1490.571077,36221.987143,NaN,110.053151,302.112000,NaN,209.645675,1.121151,NaN,249923.229256,0.141391,NaN,1.009688e+06,2.264800,NaN,202194.015531,2536.151429,NaN,1057.765667,25.028067,NaN,6709.816310,0.812400,NaN,6019.269834,111.983333,NaN,1243.860266,1833.018333,NaN,4.190290,156.5500,NaN,74.397471,0.200093,NaN,2.983796e+06,0.057478,NaN,8.988253e+05


In [8]:
# The time interval between rows before patching, you can see that there are missing values
(train_merged.index[1:]-train_merged.index[:-1]).value_counts()

timestamp
60     2136666
120         80
180         10
        ...   
360          5
300          5
480          1
Name: count, Length: 7, dtype: int64

In [9]:
# Fill in missing rows - padding method
train_merged = train_merged.reindex(range(train_merged.index[0],train_merged.index[-1]+60,60),method='pad')
# after replenishment
(train_merged.index[1:]-train_merged.index[:-1]).value_counts()

timestamp
60    2136959
Name: count, dtype: int64

In [10]:
# Reducing timestamp indexes to normal columns
train_merged = train_merged.reset_index()
train_merged

,timestamp,Close_0,Target_0,Volume_0,Close_1,Target_1,Volume_1,Close_2,Target_2,Volume_2,Close_3,Target_3,Volume_3,Close_4,Target_4,Volume_4,Close_5,Target_5,Volume_5,Close_6,Target_6,Volume_6,Close_7,Target_7,Volume_7,Close_8,Target_8,Volume_8,Close_9,Target_9,Volume_9,Close_10,Target_10,Volume_10,Close_11,Target_11,Volume_11,Close_12,Target_12,Volume_12,Close_13,Target_13,Volume_13
0,1514764860,8.530000,-0.014399,78.380000,13850.176000,-0.014643,31.550062,2374.590000,-0.004218,19.233005,NaN,NaN,NaN,NaN,NaN,NaN,7.657600,-0.013922,6626.713370,738.507500,-0.004809,335.987856,25.877000,-0.008264,121.087310,NaN,NaN,NaN,225.206667,-0.009791,411.896642,NaN,NaN,NaN,329.4600,NaN,6.635710,NaN,NaN,NaN,NaN,NaN,NaN
1,1514764920,8.514500,-0.015875,71.390000,13828.102000,-0.015037,31.046432,2372.286667,-0.004079,24.050259,NaN,NaN,NaN,NaN,NaN,NaN,7.656700,-0.014534,3277.475494,738.260000,-0.004441,232.793141,25.897000,-0.029902,1.468019,NaN,NaN,NaN,224.856667,-0.012991,3640.502706,NaN,NaN,NaN,329.4500,-0.009690,0.349420,NaN,NaN,NaN,NaN,NaN,NaN
2,1514764980,8.484800,-0.015410,1546.820000,13801.314000,-0.010309,55.061820,2372.063333,-0.002892,42.676438,NaN,NaN,NaN,NaN,NaN,NaN,7.651200,-0.012546,5623.557585,737.502500,-0.004206,174.138031,26.469500,-0.030832,76.163922,NaN,NaN,NaN,226.000000,-0.003572,328.350286,NaN,NaN,NaN,329.1900,0.006567,1.189553,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136957,1642982280,384.201667,NaN,496.467537,36276.223333,NaN,49.867700,302.430000,NaN,198.196485,1.123465,NaN,430719.654610,0.141800,NaN,7.433686e+05,2.272983,NaN,14907.125055,2541.388571,NaN,821.614133,25.152780,NaN,1334.601131,0.814233,NaN,20511.345658,112.258000,NaN,790.632440,1831.573333,NaN,2.142748,156.5125,NaN,21.597886,0.200803,NaN,7.076212e+05,0.057509,NaN,1.301079e+06
2136958,1642982340,384.002000,NaN,368.684921,36276.623333,NaN,43.030556,302.562500,NaN,76.954035,1.124831,NaN,352929.794282,0.141800,NaN,7.287195e+05,2.275100,NaN,14559.759824,2541.040000,NaN,749.548301,25.135800,NaN,2410.205214,0.815833,NaN,9064.791667,112.342400,NaN,331.208442,1833.186000,NaN,3.985169,156.8940,NaN,51.115980,0.200968,NaN,1.988516e+05,0.057521,NaN,9.245588e+05
2136959,1642982400,382.930000,NaN,1490.571077,36221.987143,NaN,110.053151,302.112000,NaN,209.645675,1.121151,NaN,249923.229256,0.141391,NaN,1.009688e+06,2.264800,NaN,202194.015531,2536.151429,NaN,1057.765667,25.028067,NaN,6709.816310,0.812400,NaN,6019.269834,111.983333,NaN,1243.860266,1833.018333,NaN,4.190290,156.5500,NaN,74.397471,0.200093,NaN,2.983796e+06,0.057478,NaN,8.988253e+05


In [11]:
# Fill the null with ffill, but limit it to 100 to prevent the error from being too large
for id in range(14):
    train_merged[f'Close_{id}'] = train_merged[f'Close_{id}'].ffill(limit=100)
    train_merged[f'Volume_{id}'] = train_merged[f'Volume_{id}'].ffill(limit=100)

In [12]:
train_merged

,timestamp,Close_0,Target_0,Volume_0,Close_1,Target_1,Volume_1,Close_2,Target_2,Volume_2,Close_3,Target_3,Volume_3,Close_4,Target_4,Volume_4,Close_5,Target_5,Volume_5,Close_6,Target_6,Volume_6,Close_7,Target_7,Volume_7,Close_8,Target_8,Volume_8,Close_9,Target_9,Volume_9,Close_10,Target_10,Volume_10,Close_11,Target_11,Volume_11,Close_12,Target_12,Volume_12,Close_13,Target_13,Volume_13
0,1514764860,8.530000,-0.014399,78.380000,13850.176000,-0.014643,31.550062,2374.590000,-0.004218,19.233005,NaN,NaN,NaN,NaN,NaN,NaN,7.657600,-0.013922,6626.713370,738.507500,-0.004809,335.987856,25.877000,-0.008264,121.087310,NaN,NaN,NaN,225.206667,-0.009791,411.896642,NaN,NaN,NaN,329.4600,NaN,6.635710,NaN,NaN,NaN,NaN,NaN,NaN
1,1514764920,8.514500,-0.015875,71.390000,13828.102000,-0.015037,31.046432,2372.286667,-0.004079,24.050259,NaN,NaN,NaN,NaN,NaN,NaN,7.656700,-0.014534,3277.475494,738.260000,-0.004441,232.793141,25.897000,-0.029902,1.468019,NaN,NaN,NaN,224.856667,-0.012991,3640.502706,NaN,NaN,NaN,329.4500,-0.009690,0.349420,NaN,NaN,NaN,NaN,NaN,NaN
2,1514764980,8.484800,-0.015410,1546.820000,13801.314000,-0.010309,55.061820,2372.063333,-0.002892,42.676438,NaN,NaN,NaN,NaN,NaN,NaN,7.651200,-0.012546,5623.557585,737.502500,-0.004206,174.138031,26.469500,-0.030832,76.163922,NaN,NaN,NaN,226.000000,-0.003572,328.350286,NaN,NaN,NaN,329.1900,0.006567,1.189553,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136957,1642982280,384.201667,NaN,496.467537,36276.223333,NaN,49.867700,302.430000,NaN,198.196485,1.123465,NaN,430719.654610,0.141800,NaN,7.433686e+05,2.272983,NaN,14907.125055,2541.388571,NaN,821.614133,25.152780,NaN,1334.601131,0.814233,NaN,20511.345658,112.258000,NaN,790.632440,1831.573333,NaN,2.142748,156.5125,NaN,21.597886,0.200803,NaN,7.076212e+05,0.057509,NaN,1.301079e+06
2136958,1642982340,384.002000,NaN,368.684921,36276.623333,NaN,43.030556,302.562500,NaN,76.954035,1.124831,NaN,352929.794282,0.141800,NaN,7.287195e+05,2.275100,NaN,14559.759824,2541.040000,NaN,749.548301,25.135800,NaN,2410.205214,0.815833,NaN,9064.791667,112.342400,NaN,331.208442,1833.186000,NaN,3.985169,156.8940,NaN,51.115980,0.200968,NaN,1.988516e+05,0.057521,NaN,9.245588e+05
2136959,1642982400,382.930000,NaN,1490.571077,36221.987143,NaN,110.053151,302.112000,NaN,209.645675,1.121151,NaN,249923.229256,0.141391,NaN,1.009688e+06,2.264800,NaN,202194.015531,2536.151429,NaN,1057.765667,25.028067,NaN,6709.816310,0.812400,NaN,6019.269834,111.983333,NaN,1243.860266,1833.018333,NaN,4.190290,156.5500,NaN,74.397471,0.200093,NaN,2.983796e+06,0.057478,NaN,8.988253e+05


In [13]:
# test_df is for training
test_df = train_merged

# Feature_Engineering

In [14]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [15]:
def MACD(df, short_term=12 * 30, long_term=26 * 30):
    '''Pass in a df and return the df with the 'MACD_id' columns'''
    tmp_col = ['EMA_short', 'EMA_long', 'DIF', 'DEA']
    for id in range(14):
        cols_to_select = [f'Close_{id}', 'timestamp', f'Target_{id}']
        df_selected = df[cols_to_select].sort_values('timestamp')

        # Calculate short-term (fast) moving averages（EMA12）
        df_selected['EMA_short'] = df_selected[f'Close_{id}'].ewm(span=short_term, adjust=False).mean()

        # Calculating long-term (slow) moving averages（EMA26）
        df_selected['EMA_long'] = df_selected[f'Close_{id}'].ewm(span=long_term, adjust=False).mean()

        # Calculation of deviation（DIF）
        df_selected['DIF'] = df_selected['EMA_short'] - df_selected['EMA_long']

        # Calculating the DEA using a 9-day exponential moving average
        df_selected['DEA'] = df_selected['DIF'].ewm(span=9, adjust=False).mean()

        # Calculating the MACD line
        df_selected[f'MACD_id{id}'] = df_selected['DIF'] - df_selected['DEA']

        # Merge back into the original DataFrame
        df = df.merge(df_selected[['timestamp', f'MACD_id{id}']], on='timestamp', how='left')

    # Check and delete temporary columns
    df.drop(columns=tmp_col, errors='ignore', inplace=True)
    return df

In [16]:
def VolumeRatio(df, mean_term=300):
    '''
    Pass in a df and return the df with the column 'Volume_Ratio_id'
    '''
    tmp_col = ['Volume_Mean']
    
    for id in range(14):
        # Calculate the 'Volume Mean' column, which represents the volume mean for the previous n days
        df[f'Volume_Mean'] = df[f'Volume_{id}'].rolling(window=mean_term).mean()
        
        # Calculate the 'Volume Ratio' column
        df[f'Volume_Ratio_id{id}'] = df[f'Volume_{id}'] / df[f'Volume_Mean']
    
    # Check and delete temporary columns
    df.drop(columns=tmp_col, errors='ignore', inplace=True)
    return df

In [17]:
def RSI(test_df, window_size=300):
    '''
    Pass in a df, return the df with the 'RSI_id' columns
    '''
    tmp_col = ['Price_Change', 'Gain', 'Loss', 'Avg_Gain', 'Avg_Loss', 'RS']
    
    for id in range(14):
        col_name = f'Close_{id}'
    
        # Calculate daily price changes
        test_df['Price_Change'] = test_df[col_name].diff()
    
        # Categorising price changes into upward and downward
        test_df['Gain'] = test_df['Price_Change'].apply(lambda x: x if x > 0 else 0)
        test_df['Loss'] = test_df['Price_Change'].apply(lambda x: -x if x < 0 else 0)
    
        # Calculation of the average of increase and decrease
        test_df['Avg_Gain'] = test_df['Gain'].rolling(window=window_size).mean()
        test_df['Avg_Loss'] = test_df['Loss'].rolling(window=window_size).mean()
    
        # Calculation of the relative strength index（RSI）
        test_df['RS'] = test_df['Avg_Gain'] / test_df['Avg_Loss']
        test_df[f'RSI_id{id}'] = 100 - (100 / (1 + test_df['RS']))
    
    test_df.drop(tmp_col, axis=1, inplace=True)
    return test_df

In [18]:
def get_features(df, train=True):   
    for id in range(14):    # 14
        for lag in lags: # 3
            # feature 1
            # Convolutional (smoothing) processing + fill in the vacant bits with 1 + roll rolls 1 to the top
            # This feature represents, for example, the logarithm of the price of asset 1 on day 60 versus the average price from day 1 - day 60
            df[f'log_close/mean_{lag}_id{id}'] = np.log( np.array(df[f'Close_{id}']) /  np.roll(np.append(np.convolve( np.array(df[f'Close_{id}']), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), lag-1)  )
            # feature 2
            # Logarithmic returns at lagged lag period for asset with id=id
            # This feature represents, for example, the logarithm of the ratio of the price of asset 1 on day 60 to the price on day 1
            df[f'log_return_{lag}_id{id}']     = np.log( np.array(df[f'Close_{id}']) /  np.roll(np.array(df[f'Close_{id}']), lag)  )
    for lag in lags: #3
        # Feature A
        # This feature represents, for example, the mean of the 60th order lag of feature 1 for all assets. 
        df[f'mean_close/mean_{lag}'] =  np.mean(df.iloc[:,df.columns.str.startswith(f'log_close/mean_{lag}_id')], axis=1)
        # Feature B
        # This feature represents, for example, the mean of the 60th order lag of feature 2 for all assets. 
        df[f'mean_log_returns_{lag}'] = np.mean(df.iloc[:,df.columns.str.startswith(f'log_return_{lag}_id')] ,    axis=1)
        for id in range(14):
            # feature 5
            # This feature represents, for example, the difference between feature 1 and feature A of asset 1
            df[f'log_close/mean_{lag}-mean_close/mean_{lag}_id{id}'] = np.array( df[f'log_close/mean_{lag}_id{id}']) - np.array( df[f'mean_close/mean_{lag}']  )
            # feature NO.6
            # This feature represents, for example, the difference between feature 2 and feature B for asset 1
            df[f'log_return_{lag}-mean_log_returns_{lag}_id{id}']    = np.array( df[f'log_return_{lag}_id{id}'])     - np.array( df[f'mean_log_returns_{lag}'] )

    for id in range(14):
        df = df.drop([f'Close_{id}'], axis=1)
        df = df.drop([f'Volume_{id}'],axis=1)

    return df

In [19]:
def replace_below_first_non_empty_900_nan(column):
    first_non_empty_index = column.first_valid_index()
    
    if first_non_empty_index is not None:
        column.loc[first_non_empty_index:first_non_empty_index+899] = np.nan
    return column

In [20]:
# For CV score calculation
def corr_score(pred, valid):
    len_data = len(pred)
    mean_pred = np.sum(pred) / len_data
    mean_valid = np.sum(valid) / len_data
    var_pred = np.sum(np.square(pred - mean_pred)) / len_data
    var_valid = np.sum(np.square(valid - mean_valid)) / len_data

    cov = np.sum((pred * valid)) / len_data - mean_pred * mean_valid
    corr = cov / np.sqrt(var_pred * var_valid)

    return corr

In [21]:
def correlation_scorer(y_true, y_pred):
    correlation, _ = np.corrcoef(y_true, y_pred)
    return correlation[1]

In [22]:
test_df

,timestamp,Close_0,Target_0,Volume_0,Close_1,Target_1,Volume_1,Close_2,Target_2,Volume_2,Close_3,Target_3,Volume_3,Close_4,Target_4,Volume_4,Close_5,Target_5,Volume_5,Close_6,Target_6,Volume_6,Close_7,Target_7,Volume_7,Close_8,Target_8,Volume_8,Close_9,Target_9,Volume_9,Close_10,Target_10,Volume_10,Close_11,Target_11,Volume_11,Close_12,Target_12,Volume_12,Close_13,Target_13,Volume_13
0,1514764860,8.530000,-0.014399,78.380000,13850.176000,-0.014643,31.550062,2374.590000,-0.004218,19.233005,NaN,NaN,NaN,NaN,NaN,NaN,7.657600,-0.013922,6626.713370,738.507500,-0.004809,335.987856,25.877000,-0.008264,121.087310,NaN,NaN,NaN,225.206667,-0.009791,411.896642,NaN,NaN,NaN,329.4600,NaN,6.635710,NaN,NaN,NaN,NaN,NaN,NaN
1,1514764920,8.514500,-0.015875,71.390000,13828.102000,-0.015037,31.046432,2372.286667,-0.004079,24.050259,NaN,NaN,NaN,NaN,NaN,NaN,7.656700,-0.014534,3277.475494,738.260000,-0.004441,232.793141,25.897000,-0.029902,1.468019,NaN,NaN,NaN,224.856667,-0.012991,3640.502706,NaN,NaN,NaN,329.4500,-0.009690,0.349420,NaN,NaN,NaN,NaN,NaN,NaN
2,1514764980,8.484800,-0.015410,1546.820000,13801.314000,-0.010309,55.061820,2372.063333,-0.002892,42.676438,NaN,NaN,NaN,NaN,NaN,NaN,7.651200,-0.012546,5623.557585,737.502500,-0.004206,174.138031,26.469500,-0.030832,76.163922,NaN,NaN,NaN,226.000000,-0.003572,328.350286,NaN,NaN,NaN,329.1900,0.006567,1.189553,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136957,1642982280,384.201667,NaN,496.467537,36276.223333,NaN,49.867700,302.430000,NaN,198.196485,1.123465,NaN,430719.654610,0.141800,NaN,7.433686e+05,2.272983,NaN,14907.125055,2541.388571,NaN,821.614133,25.152780,NaN,1334.601131,0.814233,NaN,20511.345658,112.258000,NaN,790.632440,1831.573333,NaN,2.142748,156.5125,NaN,21.597886,0.200803,NaN,7.076212e+05,0.057509,NaN,1.301079e+06
2136958,1642982340,384.002000,NaN,368.684921,36276.623333,NaN,43.030556,302.562500,NaN,76.954035,1.124831,NaN,352929.794282,0.141800,NaN,7.287195e+05,2.275100,NaN,14559.759824,2541.040000,NaN,749.548301,25.135800,NaN,2410.205214,0.815833,NaN,9064.791667,112.342400,NaN,331.208442,1833.186000,NaN,3.985169,156.8940,NaN,51.115980,0.200968,NaN,1.988516e+05,0.057521,NaN,9.245588e+05
2136959,1642982400,382.930000,NaN,1490.571077,36221.987143,NaN,110.053151,302.112000,NaN,209.645675,1.121151,NaN,249923.229256,0.141391,NaN,1.009688e+06,2.264800,NaN,202194.015531,2536.151429,NaN,1057.765667,25.028067,NaN,6709.816310,0.812400,NaN,6019.269834,111.983333,NaN,1243.860266,1833.018333,NaN,4.190290,156.5500,NaN,74.397471,0.200093,NaN,2.983796e+06,0.057478,NaN,8.988253e+05


In [23]:
# feature engineering
test_df = reduce_mem_usage(test_df)
test_df=MACD(test_df)
test_df=VolumeRatio(test_df)
test_df = RSI(test_df)
test_df=get_features(test_df)
test_df = reduce_mem_usage(test_df)

Memory usage of dataframe is 701.06 MB
Memory usage after optimization is: 224.18 MB
Decreased by 68.0%


C:\Users\严厉的父亲\AppData\Local\Temp\ipykernel_6696\3330322436.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_return_{lag}_id{id}']     = np.log( np.array(df[f'Close_{id}']) /  np.roll(np.array(df[f'Close_{id}']), lag)  )
C:\Users\严厉的父亲\AppData\Local\Temp\ipykernel_6696\3330322436.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_close/mean_{lag}_id{id}'] = np.log( np.array(df[f'Close_{id}']) /  np.roll(np.append(np.convolve( np.array(df[f'Close_{id}']), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), la

Memory usage of dataframe is 2718.64 MB
Memory usage after optimization is: 945.62 MB
Decreased by 65.2%


In [24]:
# Remove the first 900 non-null values (eliminating rows affected by the maximum lag order)
test_df.iloc[:, 15:] = test_df.iloc[:, 15:].apply(replace_below_first_non_empty_900_nan, axis=0)

In [25]:
# Delete the first 900 rows (because the targets in the first 900 rows can't be used for training because their corresponding feature values are all NaN)
test_df = test_df.iloc[900:]

In [26]:
# Selecting the data from 2021-10-25 to 2022-1-24 as being inference
test_df = test_df.loc[test_df['timestamp']>=1635091200,]
test_df = reduce_mem_usage(test_df)

Memory usage of dataframe is 59.20 MB
Memory usage after optimization is: 59.20 MB
Decreased by 0.0%


In [27]:
test_df

,timestamp,Target_0,Target_1,Target_2,Target_3,Target_4,Target_5,Target_6,Target_7,Target_8,Target_9,Target_10,Target_11,Target_12,Target_13,MACD_id0,MACD_id1,MACD_id2,MACD_id3,MACD_id4,MACD_id5,MACD_id6,MACD_id7,MACD_id8,MACD_id9,MACD_id10,MACD_id11,MACD_id12,MACD_id13,Volume_Ratio_id0,Volume_Ratio_id1,Volume_Ratio_id2,Volume_Ratio_id3,Volume_Ratio_id4,Volume_Ratio_id5,Volume_Ratio_id6,Volume_Ratio_id7,Volume_Ratio_id8,Volume_Ratio_id9,Volume_Ratio_id10,Volume_Ratio_id11,Volume_Ratio_id12,Volume_Ratio_id13,RSI_id0,RSI_id1,RSI_id2,RSI_id3,RSI_id4,RSI_id5,RSI_id6,RSI_id7,RSI_id8,RSI_id9,RSI_id10,RSI_id11,RSI_id12,RSI_id13,log_close/mean_60_id0,log_return_60_id0,log_close/mean_300_id0,log_return_300_id0,log_close/mean_900_id0,log_return_900_id0,log_close/mean_60_id1,log_return_60_id1,log_close/mean_300_id1,log_return_300_id1,log_close/mean_900_id1,log_return_900_id1,log_close/mean_60_id2,log_return_60_id2,log_close/mean_300_id2,log_return_300_id2,log_close/mean_900_id2,log_return_900_id2,log_close/mean_60_id3,log_return_60_id3,log_close/mean_300_id3,log_return_300_id3,log_close/mean_900_id3,log_return_900_id3,log_close/mean_60_id4,log_return_60_id4,log_close/mean_300_id4,log_return_300_id4,log_close/mean_900_id4,log_return_900_id4,log_close/mean_60_id5,log_return_60_id5,log_close/mean_300_id5,log_return_300_id5,log_close/mean_900_id5,log_return_900_id5,log_close/mean_60_id6,log_return_60_id6,log_close/mean_300_id6,log_return_300_id6,log_close/mean_900_id6,log_return_900_id6,log_close/mean_60_id7,log_return_60_id7,log_close/mean_300_id7,log_return_300_id7,log_close/mean_900_id7,log_return_900_id7,log_close/mean_60_id8,log_return_60_id8,log_close/mean_300_id8,log_return_300_id8,log_close/mean_900_id8,log_return_900_id8,log_close/mean_60_id9,log_return_60_id9,log_close/mean_300_id9,log_return_300_id9,log_close/mean_900_id9,log_return_900_id9,log_close/mean_60_id10,log_return_60_id10,log_close/mean_300_id10,log_return_300_id10,log_close/mean_900_id10,log_return_900_id10,log_close/mean_60_id11,log_return_60_id11,log_close/mean_300_id11,log_return_300_id11,log_close/mean_900_id11,log_return_900_id11,log_close/mean_60_id12,log_return_60_id12,log_close/mean_300_id12,log_return_300_id12,log_close/mean_900_id12,log_return_900_id12,log_close/mean_60_id13,log_return_60_id13,log_close/mean_300_id13,log_return_300_id13,log_close/mean_900_id13,log_return_900_id13,mean_close/mean_60,mean_log_returns_60,log_close/mean_60-mean_close/mean_60_id0,log_return_60-mean_log_returns_60_id0,log_close/mean_60-mean_close/mean_60_id1,log_return_60-mean_log_returns_60_id1,log_close/mean_60-mean_close/mean_60_id2,log_return_60-mean_log_returns_60_id2,log_close/mean_60-mean_close/mean_60_id3,log_return_60-mean_log_returns_60_id3,log_close/mean_60-mean_close/mean_60_id4,log_return_60-mean_log_returns_60_id4,log_close/mean_60-mean_close/mean_60_id5,log_return_60-mean_log_returns_60_id5,log_close/mean_60-mean_close/mean_60_id6,log_return_60-mean_log_returns_60_id6,log_close/mean_60-mean_close/mean_60_id7,log_return_60-mean_log_returns_60_id7,log_close/mean_60-mean_close/mean_60_id8,log_return_60-mean_log_returns_60_id8,log_close/mean_60-mean_close/mean_60_id9,log_return_60-mean_log_returns_60_id9,log_close/mean_60-mean_close/mean_60_id10,log_return_60-mean_log_returns_60_id10,log_close/mean_60-mean_close/mean_60_id11,log_return_60-mean_log_returns_60_id11,log_close/mean_60-mean_close/mean_60_id12,log_return_60-mean_log_returns_60_id12,log_close/mean_60-mean_close/mean_60_id13,log_return_60-mean_log_returns_60_id13,mean_close/mean_300,mean_log_returns_300,log_close/mean_300-mean_close/mean_300_id0,log_return_300-mean_log_returns_300_id0,log_close/mean_300-mean_close/mean_300_id1,log_return_300-mean_log_returns_300_id1,log_close/mean_300-mean_close/mean_300_id2,log_return_300-mean_log_returns_300_id2,log_close/mean_300-mean_close/mean_300_id3,log_return_300-mean_log_returns_300_id3,log_close/mean_300-mean_close/mean_300_id4,log_return_300-mean_log_returns_300_id4,lo

In [28]:
no_use_columns = [f'Target_{i}' for i in range(14)]
no_use_columns.append('timestamp')
no_use_columns.extend(FEATURES_EXCLUDE)
features = test_df.columns
features = features.drop(no_use_columns)
features = list(features)

# Model_Inference

In [30]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

df_asset_details = pd.read_csv(ASSET_DETAILS_CSV)
sum_of_weight = df_asset_details['Weight'].sum()
overall_correlation_score = 0.
correlation_list = []
# start inference
for asset_id in ASSET_ID_SELECTED_FOR_INFERENCE:
    model_path = f'filnal_new_models/model_id{asset_id}.txt'
    model = lgb.Booster(model_file=model_path)
    weight = df_asset_details[df_asset_details['Asset_ID'] == asset_id]['Weight'].values[0]
    X = test_df.loc[(test_df[f'Target_{asset_id}'] == test_df[f'Target_{asset_id}'])].loc[:,features]
    y = test_df.loc[(test_df[f'Target_{asset_id}'] == test_df[f'Target_{asset_id}'])].loc[:,f'Target_{asset_id}']
    y_pred = model.predict(X)
    correlation = correlation_scorer(y,y_pred)
    correlation_list.append(correlation)
    overall_correlation_score += (correlation*weight)/sum_of_weight
    print(f'correlation score for asset_{asset_id}: {correlation}')
print(f'overall weighted correlation score for 14 asset: {overall_correlation_score}')

correlation score for asset_0: -0.033459853129969394
correlation score for asset_1: 0.022814148586106187
correlation score for asset_2: 0.06956885626701224
correlation score for asset_3: 0.11410353691937435
correlation score for asset_4: 0.08062148578321958
correlation score for asset_5: 0.023172611895467304
correlation score for asset_6: 0.027502049861807747
correlation score for asset_7: 0.011872069194940356
correlation score for asset_8: 0.02433590333167075
correlation score for asset_9: 0.04617486610399766
correlation score for asset_10: 0.04063943839674869
correlation score for asset_11: 0.0018361484330397912
correlation score for asset_12: 0.04636054422075633
correlation score for asset_13: 0.05598820720058765
overall weighted correlation score for 14 asset: 0.038347613925340425
